In [126]:
using CSV, DataFrames, Statistics, Dates, Plots, LinearAlgebra, DecisionTree

In [127]:
trainData = CSV.read("../../data/train.csv", DataFrame)
testData = CSV.read("../../data/test.csv", DataFrame)
#first(trainData, 5)
first(testData, 5)


Row,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,String31,Int64,String3,String15,String15
1,2014,voiture_moyenne,4,"2,5",traction,manuelle
2,2014,voiture_moyenne,4,"2,5",traction,automatique
3,2014,VUS_petit,4,"2,5",traction,automatique
4,2014,VUS_petit,4,2,4x4,automatique
5,2014,voiture_sous_compacte,8,"5,8",propulsion,manuelle


In [128]:
IDS = DataFrame([1:150], [:ID])

Row,ID
,Int64
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [129]:
testData = hcat(IDS, testData)

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite
,Int64,Int64,String31,Int64,String3,String15,String15
1,1,2014,voiture_moyenne,4,"2,5",traction,manuelle
2,2,2014,voiture_moyenne,4,"2,5",traction,automatique
3,3,2014,VUS_petit,4,"2,5",traction,automatique
4,4,2014,VUS_petit,4,2,4x4,automatique
5,5,2014,voiture_sous_compacte,8,"5,8",propulsion,manuelle
6,6,2014,voiture_sous_compacte,8,5,propulsion,automatique
7,7,2014,voiture_sous_compacte,8,5,propulsion,manuelle
8,8,2014,VUS_petit,4,"2,4",4x4,automatique
9,9,2014,VUS_petit,6,"3,5",integrale,automatique


In [130]:
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => ".")) 

testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))

150-element Vector{Float64}:
 2.5
 2.5
 2.5
 2.0
 5.8
 5.0
 5.0
 2.4
 3.5
 5.2
 5.2
 5.9
 2.0
 ⋮
 1.6
 3.3
 5.0
 2.0
 2.0
 1.6
 2.0
 2.0
 3.0
 3.0
 1.5
 2.0

In [131]:
trainData[!,:volume_gaz] = trainData[!,:nombre_cylindres] .* trainData[!,:cylindree]

testData[!,:volume_gaz] = testData[!,:nombre_cylindres] .* testData[!,:cylindree]

150-element Vector{Float64}:
 10.0
 10.0
 10.0
  8.0
 46.4
 40.0
 40.0
  9.6
 21.0
 52.0
 52.0
 70.80000000000001
  8.0
  ⋮
  6.4
 19.799999999999997
 40.0
  8.0
  8.0
  6.4
  8.0
  8.0
 18.0
 18.0
  4.5
  8.0

In [132]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end


function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)

    encoded_data.cylindree = encoded_data.cylindree .- minimum(encoded_data.cylindree)
    encoded_data.volume_gaz = encoded_data.volume_gaz .- minimum(encoded_data.volume_gaz)
    encoded_data.annee = encoded_data.annee .-minimum(encoded_data.annee)
    #encoded_data = encode(encoded_data, :nombre_cylindres)
    return encoded_data
end

function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite]))
end

function normalize_column(data, column)
    data[!, column] = (data[!, column] .- mean(data[!, column])) ./ std(data[!, column])
    return data
end

function norm_cols(data)
    normalized_data = deepcopy(data)
    normalized_data = normalize_column(normalized_data, :cylindree)
    normalized_data = normalize_column(normalized_data, :volume_gaz)
    #normalized_data = normalize_column(normalized_data, :annee)
    return normalized_data
end

norm_cols (generic function with 1 method)

In [133]:
uniqueD = combine(x -> mean(x.consommation), groupby(trainData[:, Not(:annee)], [  :cylindree, :volume_gaz, :type, :transmission, :boite, :nombre_cylindres]))
rename!(uniqueD, :x1 => :consommation)

Row,cylindree,volume_gaz,type,transmission,boite,nombre_cylindres,consommation
,Float64,Float64,String31,String15,String15,Int64,Float64
1,4.4,35.2,voiture_moyenne,integrale,automatique,8,13.0036
2,2.0,8.0,VUS_petit,integrale,automatique,4,10.1294
3,3.3,19.8,voiture_compacte,propulsion,automatique,6,11.4805
4,5.0,40.0,voiture_deux_places,integrale,automatique,8,13.0672
5,1.5,4.5,voiture_minicompacte,traction,automatique,3,7.49574
6,1.5,4.5,voiture_minicompacte,traction,manuelle,3,7.58742
7,3.8,22.8,VUS_standard,integrale,automatique,6,11.2005
8,3.3,19.8,monospace,traction,automatique,6,11.7801
9,5.0,40.0,VUS_standard,4x4,automatique,8,14.7006


In [134]:
testData

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64
1,1,2014,voiture_moyenne,4,2.5,traction,manuelle,10.0
2,2,2014,voiture_moyenne,4,2.5,traction,automatique,10.0
3,3,2014,VUS_petit,4,2.5,traction,automatique,10.0
4,4,2014,VUS_petit,4,2.0,4x4,automatique,8.0
5,5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle,46.4
6,6,2014,voiture_sous_compacte,8,5.0,propulsion,automatique,40.0
7,7,2014,voiture_sous_compacte,8,5.0,propulsion,manuelle,40.0
8,8,2014,VUS_petit,4,2.4,4x4,automatique,9.6
9,9,2014,VUS_petit,6,3.5,integrale,automatique,21.0


In [135]:
testData = DataFrames.leftjoin(testData,uniqueD, on = [:type,	:nombre_cylindres, :cylindree,	:transmission,	:boite,	:volume_gaz])

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64?
1,1,2014,voiture_moyenne,4,2.5,traction,manuelle,10.0,8.11069
2,2,2014,voiture_moyenne,4,2.5,traction,automatique,10.0,7.59532
3,3,2014,VUS_petit,4,2.5,traction,automatique,10.0,8.66848
4,4,2014,VUS_petit,4,2.0,4x4,automatique,8.0,8.71148
5,6,2014,voiture_sous_compacte,8,5.0,propulsion,automatique,40.0,13.8359
6,7,2014,voiture_sous_compacte,8,5.0,propulsion,manuelle,40.0,13.8359
7,8,2014,VUS_petit,4,2.4,4x4,automatique,9.6,10.2265
8,9,2014,VUS_petit,6,3.5,integrale,automatique,21.0,10.9459
9,13,2014,VUS_petit,4,2.0,traction,automatique,8.0,9.30079


In [136]:
missingRows = testData[ismissing.(testData[!,:consommation]), :]

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64?
1,5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle,46.4,missing
2,10,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing
3,11,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing
4,12,2014,voiture_sous_compacte,12,5.9,propulsion,automatique,70.8,missing
5,17,2015,camionnette_standard,8,5.3,propulsion,automatique,42.4,missing
6,18,2015,camionnette_standard,8,5.3,propulsion,automatique,42.4,missing
7,19,2015,camionnette_standard,8,5.3,4x4,automatique,42.4,missing
8,20,2015,camionnette_standard,8,5.3,4x4,automatique,42.4,missing
9,39,2016,voiture_moyenne,4,2.0,traction,manuelle,8.0,missing


In [137]:
encoded_train = encode_data(trainData)
filtered_train = removeRows(encoded_train)
#normed_train = norm_cols(filtered_train)

encoded_test = encode_data(missingRows)
filtered_test = removeRows(encoded_test)
#normed_test = norm_cols(filtered_test)

X_train =  Matrix(filtered_train[:, Not(:consommation)])
y_train = filtered_train.consommation
X_test = Matrix(filtered_test[:, Not([:ID, :consommation])])

30×21 Matrix{Float64}:
 0.0   8.0  4.6  42.8  1.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  1.0  0.0
 0.0  10.0  4.0  48.4  0.0  1.0  0.0     0.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  10.0  4.0  48.4  0.0  1.0  0.0     0.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  12.0  4.7  67.2  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  1.0
 1.0   8.0  4.1  38.8  0.0  0.0  1.0     0.0  1.0  0.0  0.0  0.0  0.0  1.0
 1.0   8.0  4.1  38.8  0.0  0.0  1.0  …  0.0  1.0  0.0  0.0  0.0  0.0  1.0
 1.0   8.0  4.1  38.8  0.0  0.0  1.0     0.0  0.0  0.0  1.0  0.0  0.0  1.0
 1.0   8.0  4.1  38.8  0.0  0.0  1.0     0.0  0.0  0.0  1.0  0.0  0.0  1.0
 2.0   4.0  0.8   4.4  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  1.0  0.0
 2.0   6.0  2.4  18.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  1.0
 3.0   4.0  0.8   4.4  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0  1.0  0.0
 3.0   6.0  2.4  18.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  1.0
 3.0   3.0  0.0   0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  1.0  0.0  1.0
 ⋮

In [ ]:
function decisionTree_remise(X_train ,y_train,  X_test)
    model = RandomForestRegressor(n_subfeatures=6, n_trees=100,min_samples_leaf=1,min_purity_increase=0.0, max_depth=10, min_samples_split=2)
    DecisionTree.fit!(model, X_train, y_train)
    ychap = DecisionTree.predict(model, X_test)
    return ychap
end

decisionTree_remise (generic function with 1 method)

In [142]:
missingRows

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64?
1,5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle,46.4,missing
2,10,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing
3,11,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing
4,12,2014,voiture_sous_compacte,12,5.9,propulsion,automatique,70.8,missing
5,17,2015,camionnette_standard,8,5.3,propulsion,automatique,42.4,missing
6,18,2015,camionnette_standard,8,5.3,propulsion,automatique,42.4,missing
7,19,2015,camionnette_standard,8,5.3,4x4,automatique,42.4,missing
8,20,2015,camionnette_standard,8,5.3,4x4,automatique,42.4,missing
9,39,2016,voiture_moyenne,4,2.0,traction,manuelle,8.0,missing


In [143]:
res = decisionTree_remise(X_train ,y_train,  X_test)
missingRows.consommation = res

LoadError: BoundsError: attempt to access 21-element view(::Matrix{Float64}, 1, :) with eltype Float64 at index [23]

In [88]:
testData

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64?
1,100,2020,VUS_petit,4,2.0,integrale,automatique,8.0,10.0909
2,108,2020,VUS_petit,4,2.0,integrale,automatique,8.0,10.0909
3,134,2023,VUS_standard,8,5.0,4x4,automatique,40.0,14.7006
4,148,2024,voiture_sous_compacte,6,3.0,propulsion,automatique,18.0,11.0899
5,106,2020,voiture_moyenne,4,2.0,integrale,automatique,8.0,8.71148
6,26,2015,VUS_standard,8,6.2,4x4,automatique,49.6,14.7006
7,28,2015,VUS_standard,8,6.2,4x4,automatique,49.6,14.7006
8,79,2019,VUS_petit,4,2.4,integrale,automatique,9.6,9.80042
9,114,2021,VUS_petit,4,2.0,traction,automatique,8.0,7.58742


In [89]:
testData = DataFrames.leftjoin(testData,missingRows, on = [:ID, :annee,	:type,	:nombre_cylindres,	:cylindree,	:transmission,	:boite,	:volume_gaz], makeunique=true)

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation,consommation_1
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64?,Float64?
1,1,2014,voiture_moyenne,4,2.5,traction,manuelle,10.0,missing,10.6103
2,2,2014,voiture_moyenne,4,2.5,traction,automatique,10.0,missing,10.9905
3,3,2014,VUS_petit,4,2.5,traction,automatique,10.0,missing,11.3021
4,4,2014,VUS_petit,4,2.0,4x4,automatique,8.0,missing,10.1966
5,5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle,46.4,missing,11.2641
6,9,2014,VUS_petit,6,3.5,integrale,automatique,21.0,missing,12.1507
7,10,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing,12.2741
8,11,2014,voiture_deux_places,10,5.2,integrale,manuelle,52.0,missing,12.2741
9,12,2014,voiture_sous_compacte,12,5.9,propulsion,automatique,70.8,missing,11.3674


In [90]:
testData.consommation = ifelse.(ismissing.(testData.consommation), testData.consommation_1, testData.consommation)

150-element Vector{Float64}:
 10.610252355777856
 10.99051678018392
 11.302085025304883
 10.196587037268237
 11.264128699513444
 12.150737778895031
 12.274101521802768
 12.274101521802768
 11.367433559545022
 10.613431892020554
 11.388912241202272
 11.388912241202272
 12.38826192445991
  ⋮
 13.0672222222222
 13.0672222222222
  9.80041666666667
  9.80041666666667
  9.80041666666667
  8.71148148148148
 10.6913636363636
 10.6913636363636
  9.4084
 15.6806666666667
 15.6806666666667
 10.2265217391304

In [91]:
sort(testData, :ID)

Row,ID,annee,type,nombre_cylindres,cylindree,transmission,boite,volume_gaz,consommation,consommation_1
,Int64,Int64,String31,Int64,Float64,String15,String15,Float64,Float64,Float64?
1,1,2014,voiture_moyenne,4,2.5,traction,manuelle,10.0,10.6103,10.6103
2,2,2014,voiture_moyenne,4,2.5,traction,automatique,10.0,10.9905,10.9905
3,3,2014,VUS_petit,4,2.5,traction,automatique,10.0,11.3021,11.3021
4,4,2014,VUS_petit,4,2.0,4x4,automatique,8.0,10.1966,10.1966
5,5,2014,voiture_sous_compacte,8,5.8,propulsion,manuelle,46.4,11.2641,11.2641
6,6,2014,voiture_sous_compacte,8,5.0,propulsion,automatique,40.0,13.8359,missing
7,7,2014,voiture_sous_compacte,8,5.0,propulsion,manuelle,40.0,13.8359,missing
8,8,2014,VUS_petit,4,2.4,4x4,automatique,9.6,10.2265,missing
9,9,2014,VUS_petit,6,3.5,integrale,automatique,21.0,12.1507,12.1507


In [100]:
df_pred = sort(testData[:, [:ID, :consommation]], :ID)
rename!(df_pred, :ID => :id)
CSV.write("benchmark8.csv", df_pred)

"benchmark8.csv"

In [93]:
benchmark = CSV.read("../soumission_2/benchmark1.csv", DataFrame)

Row,id,consommation
,Int64,Float64
1,1,8.7731
2,2,8.07005
3,3,9.54555
4,4,9.8891
5,5,14.683
6,6,14.683
7,7,14.683
8,8,9.8891
9,9,10.9249


In [99]:
println(benchmark.consommation)
#rmse
sqrt(mean((benchmark.consommation .- df_pred.consommation).^2))

[8.773101155974718, 8.070047540540804, 9.545548529167164, 9.889100280579937, 14.682977190876372, 14.682977190876372, 14.682977190876372, 9.889100280579937, 10.924893508841617, 15.345064163165286, 15.345064163165286, 15.345064163165286, 8.840702060295868, 9.545548529167164, 8.773101155974718, 11.221135830353218, 13.463562602124174, 13.463562602124174, 13.463562602124174, 13.463562602124174, 14.138433915671541, 13.463562602124174, 13.463562602124174, 14.138433915671541, 13.463562602124174, 14.138433915671541, 13.463562602124174, 14.138433915671541, 12.897491767520393, 10.620542993264337, 11.632846725905672, 9.545548529167164, 9.889100280579937, 11.632846725905672, 9.889100280579937, 7.842333308965198, 10.620542993264337, 10.620542993264337, 8.773101155974718, 9.545548529167164, 12.897491767520393, 10.924893508841617, 9.889100280579937, 8.070047540540804, 8.773101155974718, 11.632846725905672, 9.889100280579937, 10.924893508841617, 7.4361978718638, 15.345064163165286, 9.889100280579937, 1

1.3473363784095678